In [34]:
import pandas as pd
import numpy as np

In [78]:
df_1 = pd.read_csv('/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/result_VH271613_mathbert1.csv')

In [79]:
df_1

,student_id,true_labels,predicted,score_to_predict,srace10,dsex,accom2,iep,lep
0,bOo8q8Oj0N,1,1,1,3.0,1.0,1.0,1.0,1.0
1,qm76aAIl9D,1,1,1,1.0,1.0,2.0,1.0,2.0
2,Si3MMasECV,1,1,1,3.0,2.0,1.0,1.0,2.0
3,CZVb4lFrLE,1,1,1,3.0,2.0,2.0,2.0,2.0
4,lhFUAuLxPL,1,1,1,1.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
4173,huHHC6Z1gM,1,1,1,1.0,1.0,2.0,2.0,2.0
4174,pdujpF3gGD,1,1,1,1.0,1.0,1.0,1.0,2.0
4175,dwlDM7JMK8,3,3,3,1.0,2.0,2.0,2.0,2.0
4176,j1VnE6sVDd,1,1,1,1.0,2.0,2.0,2.0,2.0


In [80]:
def calculate_standard_mean_difference(df, predicted_column, true_labels_column, col, value):
    if col == 'none' and value == -1 : filtered_df = df
    else : filtered_df = df[df[col] == value]

    # Filter the DataFrame based on srace10 == 1
    


    # Calculate the mean and standard deviation for predicted and true_labels columns
    mean_predicted = filtered_df[predicted_column].mean()
    mean_true_labels = filtered_df[true_labels_column].mean()
    std_predicted = filtered_df[predicted_column].std()
    std_true_labels = filtered_df[true_labels_column].std()

    
    # Calculate the sample sizes for predicted and true_labels columns
    n_predicted = len(filtered_df[predicted_column])
    n_true_labels = len(filtered_df[true_labels_column])


    # Calculate the standard mean difference
    numerator = mean_predicted - mean_true_labels
    denominator = (((n_predicted - 1) * std_predicted**2) + 
                   ((n_true_labels - 1) * std_true_labels**2)) / (n_predicted + n_true_labels - 2)
    smd = numerator / denominator
    
    return smd

In [82]:
import glob

unique_accessions =  ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']
# Dictionary to store the dataframes
dfs = {}

# Folder path
folder_path = '/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/Final Results/'

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the file pattern
    file_pattern = folder_path + 'result_' + accession + '*.csv'

    # Get the list of files that match the pattern
    file_list = glob.glob(file_pattern)

    # Check if any files exist
    if file_list:
        # Read the first CSV file into a dataframe
        df = pd.read_csv(file_list[0])

        # Add the dataframe to the dictionary
        dfs[accession] = df
    else:
        print(f"No files found for accession: {accession}")


In [83]:
df_1 = dfs['VH139380']

In [84]:
df_1['score_to_predict'] = df_1['true_labels']
df_1

,srace10,dsex,accom2,iep,lep,student_id,predicted,true_labels,score_to_predict
0,3.0,1.0,1.0,1.0,1.0,v65cpQmUxp,1,2,2
1,3.0,1.0,2.0,2.0,2.0,o9SYEexR1m,3,3,3
2,4.0,1.0,2.0,2.0,2.0,NjlNf7rrp8,3,3,3
3,2.0,1.0,2.0,2.0,2.0,7e8OhouIvQ,2,2,2
4,1.0,2.0,2.0,2.0,2.0,0UoYogCzaD,3,3,3
...,...,...,...,...,...,...,...,...,...
1811,1.0,1.0,2.0,2.0,2.0,gm8oBiCJKl,3,3,3
1812,1.0,2.0,2.0,2.0,2.0,S5vw4CurMa,2,2,2
1813,1.0,2.0,2.0,2.0,2.0,1p9xlu401K,1,2,2
1814,4.0,1.0,2.0,2.0,2.0,6p0uieBgtE,3,3,3


In [81]:
prompt_results = {}

smd_overall = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'none', -1)
prompt_results['Overall'] = smd_overall
smd_male = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'dsex', 1)
prompt_results['SMD Male'] = smd_male

smd_female = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'dsex', 2)
prompt_results['SMD Female'] = smd_female

smd_accom = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'accom2', 1)
prompt_results['SMD Accommodated'] = smd_accom

smd_non_accom = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'accom2', 2)
prompt_results['SMD Non-Accommodated'] = smd_non_accom

smd_lep = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'lep', 1)
prompt_results['SMD LEP'] = smd_lep

smd_non_lep = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'lep', 2)
prompt_results['SMD Non LEP'] = smd_non_lep

smd_iep = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'iep', 1)
prompt_results['SMD IEP'] = smd_iep

smd_non_iep = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'iep', 2)
prompt_results['SMD Non IEP'] = smd_non_iep

smd_while = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 1)
prompt_results['SMD White'] = smd_while

smd_black = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 2)
prompt_results['SMD Black'] = smd_black

smd_hispanic = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 3)
prompt_results['SMD Hispanic'] = smd_hispanic

smd_asian = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 4)
prompt_results['SMD Asian'] = smd_asian

smd_ai = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 5)
prompt_results['SMD American Indian'] = smd_ai

smd_pi = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 6)
prompt_results['SMD Pacific Islander'] = smd_pi

smd_other = calculate_standard_mean_difference(df_1, 'predicted', 'score_to_predict', 'srace10', 7)
prompt_results['SMD Other Race'] = smd_other

In [60]:
prompt_results

{'Overall': 0.05208506711082012,
 'SMD Male': 0.058030046788295556,
 'SMD Female': 0.04627616328537145,
 'SMD Accommodated': 0.06928239545689227,
 'SMD Non-Accommodated': 0.05113869837271549,
 'SMD LEP': 0.11300946891725,
 'SMD Non LEP': 0.045551914627138215,
 'SMD IEP': 0.055678690600871744,
 'SMD Non IEP': 0.05454828627204146,
 'SMD White': 0.03764705882352938,
 'SMD Black': 0.08869438366156056,
 'SMD Hispanic': 0.0485858947476087,
 'SMD Asian': 0.051106263633531344,
 'SMD American Indian': 0.0,
 'SMD Pacific Islander': -0.5217391304347825,
 'SMD Other Race': 0.16547920018386603}

In [85]:
import pandas as pd

# Create an empty dataframe
results_df = pd.DataFrame(columns=unique_accessions)

# Loop through the unique accessions
for prompts in unique_accessions:
    df = dfs[str(prompts)]
    prompt_results = {}
    prompt_results['Prompt'] = prompts

    print(prompts)   
    smd_overall = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'none', -1)
    prompt_results['Overall'] = smd_overall
    smd_male = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'dsex', 1)
    prompt_results['SMD Male'] = smd_male
    
    smd_female = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'dsex', 2)
    prompt_results['SMD Female'] = smd_female
    
    smd_accom = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'accom2', 1)
    prompt_results['SMD Accommodated'] = smd_accom
    
    smd_non_accom = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'accom2', 2)
    prompt_results['SMD Non-Accommodated'] = smd_non_accom
    
    smd_lep = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'lep', 1)
    prompt_results['SMD LEP'] = smd_lep
    
    smd_non_lep = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'lep', 2)
    prompt_results['SMD Non LEP'] = smd_non_lep
    
    smd_iep = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'iep', 1)
    prompt_results['SMD IEP'] = smd_iep
    
    smd_non_iep = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'iep', 2)
    prompt_results['SMD Non IEP'] = smd_non_iep

    smd_while = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 1)
    prompt_results['SMD White'] = smd_while
    
    smd_black = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 2)
    prompt_results['SMD Black'] = smd_black
    
    smd_hispanic = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 3)
    prompt_results['SMD Hispanic'] = smd_hispanic
    
    smd_asian = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 4)
    prompt_results['SMD Asian'] = smd_asian
    
    smd_ai = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 5)
    prompt_results['SMD American Indian'] = smd_ai
    
    smd_pi = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 6)
    prompt_results['SMD Pacific Islander'] = smd_pi
    
    smd_other = calculate_standard_mean_difference(df, 'predicted', 'score_to_predict', 'srace10', 7)
    prompt_results['SMD Other Race'] = smd_other
    
    # Append the prompt results to the dataframe
    results_df = pd.concat([results_df, pd.DataFrame(prompt_results, index=[0])], ignore_index=True)

# Print the results dataframe
print(results_df)




VH134067
VH139380
VH266015
VH266510
VH269384
VH271613
VH302907
VH304954
VH507804
VH525628
  VH134067 VH139380 VH266015 VH266510 VH269384 VH271613 VH302907 VH304954   
0      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN  \
1      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
3      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
5      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
6      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
7      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
8      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
9      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

  VH507804 VH525628  ... SMD Non LEP   SMD IEP  SMD Non IEP  S

In [56]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.4 MB/s eta 0:00:00a 0:00:01


In [86]:
results_df.to_excel('smd_results_5.xlsx', index=False)

In [87]:
import os
import shutil

In [88]:


unique_accessions = ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']

# Source folder containing the files to be copied
source_folder = "/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/Final Results/"

# Create directories and copy files
for accession in unique_accessions:
    # Create directory if it doesn't exist
    directory = os.path.join(".", accession)
    os.makedirs(directory, exist_ok=True)
    
    # Get the files containing the accession name
    matching_files = [file for file in os.listdir(source_folder) if accession in file]
    
    # Copy the matching files to the directory
    for file in matching_files:
        source_file = os.path.join(source_folder, file)
        destination_file = os.path.join(directory, file)
        shutil.copy2(source_file, destination_file)

In [89]:
import glob

unique_accessions =  ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']
# Dictionary to store the dataframes
dfs_test = {}

# Folder path
folder_path = '/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/Final Results/'

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the file pattern
    file_pattern = folder_path + 'test_result_' + accession + '*.csv'

    # Get the list of files that match the pattern
    file_list = glob.glob(file_pattern)

    # Check if any files exist
    if file_list:
        # Read the first CSV file into a dataframe
        df = pd.read_csv(file_list[0])

        # Add the dataframe to the dictionary
        dfs_test[accession] = df
    else:
        print(f"No files found for accession: {accession}")

In [94]:
import glob

unique_accessions =  ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']
# Dictionary to store the dataframes
dfs_test = {}
value_counts_test = {}
# Folder path
folder_path = '/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/Final Results/'

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the file pattern
    file_pattern = folder_path + 'test_result_' + accession + '*.csv'

    # Get the list of files that match the pattern
    file_list = glob.glob(file_pattern)

    # Check if any files exist
    if file_list:
        # Read the first CSV file into a dataframe
        df = pd.read_csv(file_list[0])
        value_counts_test[accession] = df.score_to_predict.value_counts(normalize=True)

        # Add the dataframe to the dictionary
        dfs_test[accession] = df
    else:
        print(f"No files found for accession: {accession}")

In [95]:
value_counts_test

{'VH134067': score_to_predict
 1    0.685925
 2    0.314075
 Name: proportion, dtype: float64,
 'VH139380': score_to_predict
 3    0.440535
 2    0.412289
 1    0.147175
 Name: proportion, dtype: float64,
 'VH266015': score_to_predict
 1    0.807920
 2    0.190898
 3    0.001182
 Name: proportion, dtype: float64,
 'VH266510': score_to_predict
 1    0.764199
 3    0.216248
 2    0.019553
 Name: proportion, dtype: float64,
 'VH269384': score_to_predict
 1    0.804892
 3    0.140501
 2    0.054608
 Name: proportion, dtype: float64,
 'VH271613': score_to_predict
 1    0.955074
 3    0.032321
 2    0.012605
 Name: proportion, dtype: float64,
 'VH302907': score_to_predict
 1    0.806178
 2    0.193822
 Name: proportion, dtype: float64,
 'VH304954': score_to_predict
 2    0.440029
 1    0.345972
 3    0.213999
 Name: proportion, dtype: float64,
 'VH507804': score_to_predict
 1    0.744574
 3    0.181413
 2    0.074012
 Name: proportion, dtype: float64,
 'VH525628': score_to_predict
 1    0.72

In [98]:
df_value_count_test = pd.DataFrame(value_counts_test)

In [99]:
df_value_count_test

,VH134067,VH139380,VH266015,VH266510,VH269384,VH271613,VH302907,VH304954,VH507804,VH525628
score_to_predict,,,,,,,,,,
1,0.685925,0.147175,0.807920,0.764199,0.804892,0.955074,0.806178,0.345972,0.744574,0.723451
2,0.314075,0.412289,0.190898,0.019553,0.054608,0.012605,0.193822,0.440029,0.074012,0.160398
3,NaN,0.440535,0.001182,0.216248,0.140501,0.032321,NaN,0.213999,0.181413,0.116150


In [ ]:
import glob

unique_accessions =  ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']
# Dictionary to store the dataframes
dfs_test = {}
value_counts = {}
# Folder path
folder_path = '/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/Final Results/'

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the file pattern
    file_pattern = folder_path + 'test_result_' + accession + '*.csv'

    # Get the list of files that match the pattern
    file_list = glob.glob(file_pattern)

    # Check if any files exist
    if file_list:
        # Read the first CSV file into a dataframe
        df = pd.read_csv(file_list[0])
        value_counts[accession] = df.score_to_predict.value_counts(normalize=True)

        # Add the dataframe to the dictionary
        dfs_test[accession] = df
    else:
        print(f"No files found for accession: {accession}")

In [96]:
import glob

unique_accessions =  ['VH134067', 'VH139380', 'VH266015', 'VH266510', 'VH269384',
                     'VH271613', 'VH302907', 'VH304954', 'VH507804', 'VH525628']
# Dictionary to store the dataframes
dfs_main = {}
value_counts= {}
# Folder path

# Loop through the unique accessions
for accession in unique_accessions:
    # Create the file pattern
    file_pattern = 'df_clean' + accession + '.csv'

    # Get the list of files that match the pattern
    file_list = glob.glob(file_pattern)

    # Check if any files exist
    if file_list:
        # Read the first CSV file into a dataframe
        df = pd.read_csv(file_list[0])
        value_counts[accession] = df.score_to_predict.value_counts(normalize=True)

        # Add the dataframe to the dictionary
       # dfs_test[accession] = df
    else:
        print(f"No files found for accession: {accession}")

/var/folders/y_/93fvbts118bgkwysz7wjp5p80000gn/T/ipykernel_93890/2382487208.py:21: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_list[0])


In [97]:
value_counts

{'VH134067': score_to_predict
 1    0.713475
 2    0.286525
 Name: proportion, dtype: float64,
 'VH139380': score_to_predict
 2    0.414906
 3    0.410211
 1    0.174883
 Name: proportion, dtype: float64,
 'VH266015': score_to_predict
 1    0.811567
 2    0.187022
 3    0.001411
 Name: proportion, dtype: float64,
 'VH266510': score_to_predict
 1    0.770654
 3    0.200100
 2    0.029246
 Name: proportion, dtype: float64,
 'VH269384': score_to_predict
 1    0.797184
 3    0.139555
 2    0.063261
 Name: proportion, dtype: float64,
 'VH271613': score_to_predict
 1    0.957371
 3    0.032169
 2    0.010460
 Name: proportion, dtype: float64,
 'VH302907': score_to_predict
 1    0.804566
 2    0.195434
 Name: proportion, dtype: float64,
 'VH304954': score_to_predict
 2    0.455740
 1    0.362823
 3    0.181437
 Name: proportion, dtype: float64,
 'VH507804': score_to_predict
 1    0.744129
 3    0.202884
 2    0.052987
 Name: proportion, dtype: float64,
 'VH525628': score_to_predict
 1    0.71

In [100]:
df_value_count_all = pd.DataFrame(value_counts)

In [101]:
df_value_count_all

,VH134067,VH139380,VH266015,VH266510,VH269384,VH271613,VH302907,VH304954,VH507804,VH525628
score_to_predict,,,,,,,,,,
1,0.713475,0.174883,0.811567,0.770654,0.797184,0.957371,0.804566,0.362823,0.744129,0.711873
2,0.286525,0.414906,0.187022,0.029246,0.063261,0.010460,0.195434,0.455740,0.052987,0.191129
3,NaN,0.410211,0.001411,0.200100,0.139555,0.032169,NaN,0.181437,0.202884,0.096998


In [102]:
val = pd.concat([df_value_count_all, df_value_count_test], axis=0)

In [104]:
val.to_excel('value_counts.xlsx')

In [90]:
dfs_test

{'VH134067':       student_id accession  score_to_predict   
 0     00NK0MwpPa  VH134067                 2  \
 1     00eCiccn2b  VH134067                 1   
 2     013BzqWhJ7  VH134067                 1   
 3     01x13bb8Ju  VH134067                 2   
 4     03Hq2FO3Wu  VH134067                 2   
 ...          ...       ...               ...   
 4478  ztIT6rU6Mb  VH134067                 1   
 4479  zwUY5ifIdo  VH134067                 1   
 4480  zxlVCqHvxV  VH134067                 2   
 4481  zxn3ptbIac  VH134067                 1   
 4482  zy4wkBcHpq  VH134067                 1   
 
                                            predict_from  year  srace10  dsex   
 0                                                    ×6  2017        3   1.0  \
 1                                         MULTIPLY BY 3  2017        3   1.0   
 2           at some point they meet with multiplication  2017        1   1.0   
 3     The relationship beetween the input numbers an...  2017        1   

In [91]:
# Concatenate dataframes
concatenated_df = pd.concat(dfs_test.values(), ignore_index=True)

# Sort dataframe by accession in ascending order
concatenated_df.sort_values(by='accession', inplace=True)

# Print the concatenated dataframe
print(concatenated_df)

       student_id accession  score_to_predict   
0      00NK0MwpPa  VH134067                 2  \
2980   BmIn9EJsNz  VH134067                 1   
2981   BmyzJKwDZ6  VH134067                 1   
2982   BnJkcuSQ6R  VH134067                 1   
2983   Bpk1eoK3cG  VH134067                 1   
...           ...       ...               ...   
26717  KbAh1Uwoe3  VH525628                 2   
26716  KQkZKlkyd0  VH525628                 1   
26715  KPNEnYEUxq  VH525628                 1   
26741  LWbw7F6Mvf  VH525628                 1   
27929  zya8Ka05Lw  VH525628                 1   

                                            predict_from  year  srace10  dsex   
0                                                     ×6  2017        3   1.0  \
2980   You can multiply two inputs and get an output....  2019        1   2.0   
2981   the nimbers are going in order but no 1 2 just...  2019        2   1.0   
2982                                         the moltpis  2019        1   2.0   
2983  

In [92]:
concatenated_df.to_csv('/Users/chandramaniyadav/Downloads/NAEP_Competition/NAEP_Math_Challenge_Datasetv1.0/test_result_final_4.csv', index=False)

In [93]:
concatenated_df['score_to_predict'].value_counts()

score_to_predict
1    19418
2     5223
3     3289
Name: count, dtype: int64

In [66]:
df_test = pd.read_csv('df_test.csv')
df_test

/var/folders/y_/93fvbts118bgkwysz7wjp5p80000gn/T/ipykernel_17229/473654489.py:1: DtypeWarning: Columns (20,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('df_test.csv')


,student_id,accession,score_to_predict,predict_from,year,srace10,dsex,accom2,iep,lep,...,eliminated2,eliminated3,eliminated4,selected1.1,selected2.1,eliminated1.1,eliminated2.1,partA_response_val,partB_response_val,partB_eliminations
0,00NK0MwpPa,VH134067,NaN,×6,2017,3,1.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00eCiccn2b,VH134067,NaN,MULTIPLY BY 3,2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,013BzqWhJ7,VH134067,NaN,at some point they meet with multiplication,2017,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01x13bb8Ju,VH134067,NaN,The relationship beetween the input numbers an...,2017,1,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03Hq2FO3Wu,VH134067,NaN,"three goes into eighteen six times, four goes ...",2017,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27925,zujaRhVvB5,VH525628,NaN,6667π,2019,2,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27926,zxEYvBEaKl,VH525628,NaN,If z and y are the smallest integers then subt...,2019,1,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27927,zxFKk3hf6Z,VH525628,NaN,beacuse the it will be confined as one also so...,2019,3,1.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27928,zxyqTqJlwS,VH525628,NaN,I know im kinda correct because if w>x>y>y is ...,2019,3,2.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
